In [1]:
import walle_rl.agents.ppo
from walle_rl.architecture.mlp import MLP
import gym
import flax.linen as nn
from flax import struct
from walle_rl.common.random import PRNGSequence
from walle_rl.buffer.buffer import GenericBuffer
import jax.numpy as jnp
import numpy as np
import jax
from stable_baselines3.common.env_util import make_vec_env
from walle_rl.architecture.model import Model
import optax

In [16]:
N=1_000
jaxa = jnp.ones(10_000_000)
npa=np.ones(10_000_000)
@jax.jit
def jit_arr(N):
    return jnp.ones(10_000_000)
@jax.jit
def bench_jax_scan(a):
    def f(i,x):
        x = x.at[i].set(i)
        return x
    a = jax.lax.fori_loop(0,N,f,a)
    return a
@jax.jit
def bench_jax(a, r):
    def f(i,x):
        x = x.at[i].set(x[i-1] * r)
        return x
    y = jax.lax.fori_loop(1,N,f,a)
    return y

@jax.jit
def compute_scan(a, r):
    def f(c,s):
        res = c * r
        return res, res
    _,res = jax.lax.scan(f, a[0]/r, jnp.arange(N))
    return res
# def bench_jax_scan(a, r):
#     a[:N] = compute_scan(jnp.array(a), r)
#     return a

In [11]:
jax_a_jit = jit_arr(N)

In [17]:
%time bench_jax_scan(jax_a_jit)
%timeit b=bench_jax_scan(jax_a_jit)

CPU times: user 39.7 ms, sys: 1.72 ms, total: 41.4 ms
Wall time: 42.1 ms
4.47 ms ± 43.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
jax_a_jit

DeviceArray([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [82]:
%time bench_jax_scan(npa,1.005)
%timeit b=bench_jax_scan(npa,1.005)

CPU times: user 54.1 ms, sys: 1.24 ms, total: 55.3 ms
Wall time: 53.7 ms
8.58 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
bench_jax_scan(jaxa,1.005)

DeviceArray([1.      , 1.005   , 1.010025, ..., 1.      , 1.      ,
             1.      ], dtype=float32)

In [66]:
buffer_cfg=dict(
            act_buf = ((), np.int32),
            adv_buf = ((), np.float32),
            rew_buf = ((), np.float32),
            ret_buf = ((), np.float32),
            val_buf = ((), np.float32),
            logp_buf = ((), np.float32),
            done_buf = ((), np.bool8)
        )
buffer = GenericBuffer(
    20000, n_envs=4, config=buffer_cfg
)

In [158]:
from typing import Dict
import chex
from functools import partial
Array=chex.Array
@struct.dataclass
class BufferJax:
    buffers: Dict[str, Array]
    ptr: int
    buffer_size: int
    batch_idx: int
    @classmethod
    def create(
        cls,
        buffer_size: int,
        n_envs: int,
        config,
    ):
        is_dict = dict()
        buffers = dict()
        for k in config.keys():
            shape, dtype = config[k]
            k_dict = False
            if isinstance(shape, dict):
                k_dict = True
            is_dict[k] = k_dict
            if k_dict:
                buffers[k] = dict()
                for part_key in shape.keys():
                    buffers[k][part_key] = jnp.zeros((buffer_size, n_envs) + shape[part_key], dtype=dtype[part_key])
            else:
                buffers[k] = jnp.zeros((buffer_size, n_envs) + shape, dtype=dtype)
        ptr, path_start_idx, max_size = 0, [0]*n_envs, buffer_size
        return cls(
            batch_idx=0,
            buffers=buffers,
            ptr=ptr,
            buffer_size=buffer_size
        )
    
    # @partial(jax.jit, static_argnames=['self'])
    def store(self, **kwargs):
        """
        store one timestep of agent-environment interaction to the buffer. If full, replaces the oldest entry
        """
        buffers = self.buffers.copy()
        # for k in self.buffers.keys():
        #     if k in kwargs:
        #         data = kwargs[k]
        #         if isinstance(data, dict):
        #             for data_k in data.keys():
        #                 d = jnp.array(data[data_k])
        #                 d = d.reshape(self.buffers[k][data_k][self.ptr].shape)
        #                 buffers[k][data_k]=self.buffers[k][data_k].at[self.ptr].set(d)
        #         else:
        #             d = jnp.array(data)
        #             d = d.reshape(self.buffers[k][self.ptr].shape)
        #         buffers[k]=self.buffers[k].at[self.ptr].set(d)
        # ptr = self.ptr + 1
        # if ptr == self.buffer_size:
        #     # wrap pointer around to start replacing items
        #     self.full = True
        #     self.ptr = 0
        return self.replace(
            buffers=buffers
        )
        

In [159]:
jaxbuf = BufferJax.create(20000, n_envs=4, config=buffer_cfg)

In [160]:
jaxbuf.buffers['act_buf']

DeviceArray([[0, 0, 0, 0],
             [0, 0, 0, 0],
             [0, 0, 0, 0],
             ...,
             [0, 0, 0, 0],
             [0, 0, 0, 0],
             [0, 0, 0, 0]], dtype=int32)

In [161]:
jaxbuf=jaxbuf.store(act_buf=jnp.ones(4, dtype=int))

In [162]:
jaxbuf.buffers['act_buf']

DeviceArray([[0, 0, 0, 0],
             [0, 0, 0, 0],
             [0, 0, 0, 0],
             ...,
             [0, 0, 0, 0],
             [0, 0, 0, 0],
             [0, 0, 0, 0]], dtype=int32)